In [152]:
import pandas as pd

In [153]:
df = pd.read_csv("Daten/masterdata_2024.csv", index_col = 0, parse_dates=True)

In [154]:
df

,Unnamed: 0,QN_2,V_TE002,V_TE005,V_TE010,V_TE020,V_TE050,V_TE100,QN_8,P,...,zeit,ecoli,entro,pos_neg,Elektrische Leitfähigkeit bei ...Grad Celsius,Sauerstoff,Temperatur,"Trübung, quantitativ",pH-Wert bei ...Grad Celsius,Wassertemperatur Hofen
2024-05-06 09:00:00,0,3.0,-999.0,17.7,15.5,14.4,14.5,13.1,3.0,1008.3,...,NaN,NaN,NaN,NaN,764.5,7,14.5,23,7.9,15.0
2024-05-06 10:00:00,1,3.0,-999.0,17.9,16.4,14.9,14.4,13.1,3.0,1008.3,...,NaN,NaN,NaN,NaN,764.5,7,14.5,23,7.9,15.0
2024-05-06 11:00:00,2,3.0,-999.0,19.2,17.3,15.3,14.4,13.1,3.0,1008.6,...,NaN,NaN,NaN,NaN,764.5,7,14.5,23,7.9,15.0
2024-05-06 12:00:00,3,3.0,-999.0,19.4,17.9,15.8,14.3,13.1,3.0,1008.8,...,NaN,NaN,NaN,NaN,764.5,7,14.5,23,7.9,15.0
2024-05-06 13:00:00,4,3.0,-999.0,18.7,18.0,16.2,14.3,13.1,3.0,1009.0,...,NaN,NaN,NaN,NaN,764.5,7,14.5,23,7.9,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-21 03:00:00,4026,3.0,-999.0,9.6,10.7,12.0,13.1,13.7,3.0,1026.5,...,NaN,NaN,NaN,NaN,885.9,10.4,12.2,8.5,8.1,14.0
2024-10-21 04:00:00,4027,3.0,-999.0,9.2,10.3,11.8,13.1,13.7,3.0,1026.8,...,NaN,NaN,NaN,NaN,885.9,10.4,12.2,8.5,8.1,14.0
2024-10-21 05:00:00,4028,3.0,-999.0,8.9,10.0,11.6,13.1,13.7,3.0,1027.0,...,NaN,NaN,NaN,NaN,885.9,10.4,12.2,8.5,8.1,14.0
2024-10-21 06:00:00,4029,3.0,-999.0,8.9,9.8,11.3,13.0,13.8,3.0,1027.1,...,NaN,NaN,NaN,NaN,885.9,10.4,12.2,8.5,8.1,14.0


In [156]:
df = df.apply(pd.to_numeric, errors = "coerce")

In [157]:
df.isna().sum()

Unnamed: 0                                          0
QN_2                                              102
V_TE002                                           102
V_TE005                                           102
V_TE010                                           102
V_TE020                                           102
V_TE050                                           102
V_TE100                                           102
QN_8                                                0
P                                                   0
P0                                                  0
QN_8_y                                            286
ABSF_STD                                          286
VP_STD                                            286
TF_STD                                            286
P_STD                                             286
TT_STD                                            286
RF_STD                                            286
TD_STD                      

In [158]:
df.drop(columns=[ "pos_neg"], inplace = True) 

In [159]:
categorical = ["WRTR", "RS_IND"]

In [160]:
df.drop(columns = categorical, inplace = True)

In [161]:
df = df.dropna(how = "all", axis = 1)

In [162]:
df = df[pd.notna(df["ecoli"])]

In [163]:
df["ecoli"] = df["ecoli"] < 1000
df["entro"] = df["entro"] < 400

In [164]:
df.dropna(how = "all", axis = 1)

,Unnamed: 0,QN_2,V_TE002,V_TE005,V_TE010,V_TE020,V_TE050,V_TE100,QN_8,P,...,QN_8_y.3,V_N,ecoli,entro,Elektrische Leitfähigkeit bei ...Grad Celsius,Sauerstoff,Temperatur,"Trübung, quantitativ",pH-Wert bei ...Grad Celsius,Wassertemperatur Hofen
2024-05-13 08:00:00,167,3.0,-999.0,20.3,18.0,16.9,16.8,14.0,3.0,1014.2,...,3.0,7.0,True,True,NaN,NaN,NaN,NaN,NaN,15.0
2024-05-21 08:00:00,359,3.0,-999.0,15.8,15.9,16.4,16.7,14.9,3.0,1007.3,...,3.0,7.0,False,False,652.3,9.7,14.9,59.7,7.9,14.0
2024-05-27 08:00:00,503,3.0,-999.0,19.0,18.1,17.9,17.4,15.4,3.0,1019.0,...,3.0,8.0,False,False,680.0,8.7,13.8,13.3,7.9,14.0
2024-06-03 10:00:00,673,3.0,-999.0,17.1,16.9,16.5,16.3,15.5,3.0,1018.9,...,3.0,8.0,False,False,NaN,NaN,NaN,NaN,NaN,14.0
2024-06-10 08:00:00,839,3.0,-999.0,22.5,20.2,19.6,19.8,17.1,3.0,1008.7,...,3.0,0.0,False,False,NaN,NaN,NaN,NaN,NaN,14.0
2024-06-17 07:00:00,1006,3.0,-999.0,19.0,18.7,19.0,18.9,17.1,3.0,1013.4,...,3.0,8.0,False,True,NaN,NaN,NaN,NaN,NaN,14.0
2024-06-28 07:00:00,1270,3.0,-999.0,23.2,22.1,22.3,22.9,19.7,3.0,1014.5,...,3.0,7.0,False,False,NaN,NaN,NaN,NaN,NaN,20.0
2024-07-02 08:00:00,1367,3.0,-999.0,21.2,19.4,19.4,21.5,20.3,3.0,1016.5,...,3.0,7.0,True,True,NaN,NaN,NaN,NaN,NaN,20.0
2024-07-09 09:00:00,1536,3.0,-999.0,27.3,23.7,21.2,20.5,18.7,3.0,1015.5,...,3.0,2.0,True,True,NaN,NaN,NaN,NaN,NaN,20.0
2024-07-15 09:00:00,1680,3.0,-999.0,26.7,23.4,21.3,21.4,19.8,3.0,1012.2,...,3.0,0.0,False,True,NaN,NaN,NaN,NaN,NaN,20.0


In [165]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Prepare the data
X = df.drop(columns=["ecoli", "entro"])
y = df["ecoli"]

# Initialize the logistic regression model
model = LogisticRegression(max_iter=1000)

# Perform cross-validation
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Print the cross-validation scores
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1201, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 650, in _validate_data
    X, y = check_X_y(X